<a href="https://colab.research.google.com/github/Xami-20/IBD_prediction/blob/main/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Packages**

## **Required installation**

In [ ]:
# Screed is a Python library for reading in FASTA/FASTQ file
# File can be uncompressed, gzipped, or bz2-zipped
!python -m pip install screed

## **Importing Packages**

In [ ]:
import os
import screed # To read Sequence files
from itertools import product
import pandas as pd

# **Extracting the Consensus sequence from the fasta files**

In [ ]:
data_path = "./PRJEB13679_fa/"
os.chdir(data_path)
# print(os.getcwd())
data_file = "./consensus/consensus_data.fa"

output = open(data_file,'w')
for i in sorted(os.listdir()):
    src_file = i
    profile = {"A": [], "T": [], "C": [], "G": []}
    reads = []
    with open(src_file,'r') as file:
        for line in file:
            if line.startswith(">")== False:
                line = line.strip()
                reads.append(line)
    del line    
    for seq in reads:
        for base in range(len(seq)):
            if seq[base].upper() == "A":
                if len(profile["A"]) <= base:
                    profile["A"].append(1)
                    profile["T"].append(0)
                    profile["C"].append(0)
                    profile["G"].append(0)
                elif len(profile["A"]) > base:
                    profile["A"][base] += 1
        
            elif seq[base].upper() == "T":
                if len(profile["T"]) <= base:
                    profile["A"].append(0)
                    profile["T"].append(1)
                    profile["C"].append(0)
                    profile["G"].append(0)
                elif len(profile["T"]) > base:
                    profile["T"][base] += 1
        
            elif seq[base].upper() == "C":
                if len(profile["C"]) <= base:
                    profile["A"].append(0)
                    profile["T"].append(0)
                    profile["C"].append(1)
                    profile["G"].append(0)
                elif len(profile["C"]) > base:
                    profile["C"][base] += 1
        
            elif seq[base].upper() == "G":
                if len(profile["G"]) <= base:
                    profile["A"].append(0)
                    profile["T"].append(0)
                    profile["C"].append(0)
                    profile["G"].append(1)
                elif len(profile["G"]) > base:
                    profile["G"][base] += 1
        
    del seq,reads
    scores = [0]*len(profile["A"])
    matrix = profile.values()
    for base in matrix:
        for index in range(len(scores)):
            if base[index] > scores[index]:
                scores[index] = base[index]
    read = ""
    for index in range(len(scores)):
        if profile["A"][index] == scores[index]:
            read += "A"
        elif profile["T"][index] == scores[index]:
            read += "T"
        elif profile["C"][index] == scores[index]:
            read += "C"
        elif profile["G"][index] == scores[index]:
            read += "G"
    if len(read) < len(scores):
        print("\n\n\t***\tMissing bases\t***\t !!!")
        print("\n\t***\t{}\t***\t !!!".format(src_file))
    
    del scores, profile
    
    sample = src_file[:src_file.find(".fa")]
    output.write(">{}\n{}\n".format(sample,read))
    
    del sample,read
#     print(data)
#     break
output.close()

# **Kmers**

## **Kmer extraction and encoding functions**

In [ ]:
def get_K_mer(seq, k_size):
    # Function to get K-mer from Sequence
    k_mers = []
    num_kmers = len(seq) - k_size + 1
    for i in range(num_kmers):
        k_mer = seq[i:i + k_size]
        k_mers.append(k_mer)
    return k_mers

def seq2binary(kmer):
    binary = ""
    for i in kmer:
        i = i.upper()
        if i == "A":
            binary += "00"
        elif i == "T":
            binary += "01"
        elif i == "C":
            binary += "10"
        elif i == "G":
            binary += "11"
    return binary

def binary2decimal(binary):
    try:
        x = int(binary,2)
        return x
    except:
        return (-1)

def seq_file2kmers(file, k_size):
    Seq_kmers = {}
    for record in screed.open(file):
        seq = record.sequence
        name = record.name
        kmers = get_K_mer(seq, k_size)
        for mer in range(len(kmers)):
            kmers[mer] = binary2decimal(seq2binary(kmers[mer]))
        underscore = name.find("_")
        if underscore > 1:
            n = name[:underscore]
        else:
            n = name
        if n in list(Seq_kmers.keys()):
            Seq_kmers[n].extend(kmers)
        else:
            Seq_kmers[n] = kmers # Dictinary of the Sequence names and their respective k-mers
    return Seq_kmers


## **kmer Reference**

In [ ]:
Dbases = "ATCG"

for kmer in range(4,15):
    path = "./possible_kmers/" + str(kmer) + "-mer_reference.csv"
    with open(path,"w") as file:
        file.write("Decimal Value,{}-mers".format(str(kmer)))
        for i in product(Dbases, repeat = kmer):
            binary = ""
            mer = ''.join(i)
            decimal = str(binary2decimal(seq2binary(mer)))
            file.write("\n{},{}".format(decimal,str(mer)))
    print('{}'.format(kmer))

## **Generating kmers from a consensus data file**

In [ ]:
src_file = "./PRJEB13679_fa/consensus/consensus_data.fa"

for k in range(4,26):
    print('\n\t\t\t\t\t\t{}\n\n'.format(k))
    path = "./kmers/" + str(k) + "-mers_decimal_PRJEB13679.csv"
    with open(path,"w") as file:
        x = seq_file2kmers(src_file,k)
        counter = 0
        l = len(x.keys())
        for key, val in x.items():
            counter += 1
            file.write(str(key))
            print("{:.2f} %".format((counter*100)/l))
            for element in val:
                file.write(",{}".format(str(element)))
            file.write("\n")    

# **Phenotypic features**

## **Generating feature files**

In [ ]:
df = pd.read_csv('./pheno_features/1359_PRJEB13679_attributes.csv')
# df = df[["library_name","total_spots","total_size","run_total_spots","run_total_bases","age","biopsy_location","body_site","diagnosis","disease_extent","disease_stat","diseasesubtype","elevation","gastrointest_disord","inflammationstatus","latitude","longitude","race","sex"]]
df = df[["library_name","biopsy_location","body_site","sex","race","age","diagnosis","gastrointest_disord","disease_extent","disease_stat","diseasesubtype","inflammationstatus","latitude","longitude"]]
df = df.drop(columns=["diagnosis"])
df.rename(columns = {'gastrointest_disord':'diagnosis',"library_name":"sample"}, inplace = True)
df.to_csv('./pheno_features/1359_features.csv', index=False)

# Extracting Labels and sex only 
labels = df[["sample","diagnosis","sex"]]
labels.to_csv('./pheno_features/1359_labels.csv', index=False)

# **Adding Labels to Kmer files**

In [ ]:
labels = pd.read_csv('./pheno_features/1359_labels.csv')
labels = labels.sort_values("sample")

for i in sorted(os.listdir("./kmers")):
    print(i)
    if i.endswith("decimal_PRJEB13679.csv"):
        kmers = "./kmers/" + i
        df = pd.read_csv(kmers,header=None)
        columns = ["sample"]
        for index in range(1,df.shape[1]):
            columns.append("kmer_feature_{}".format(index))
        # print(df.shape)
        # print(columns)
        # print(len(columns))
        df.columns = columns # adding data header
        del columns
        out = pd.merge(labels, df, on = "sample")
        del df
        out = out.drop(columns=["sex"])
        save = "./kmers/labeled/" + i
        out.to_csv(save,index=False)
        # break
    
# print(df.head(3))
# print(labels.head(3))
# print(out.head(3))
# print(df.shape[1])